In [1]:
# Imports

# Data import and manipulation
import pandas as pd
# Math
import numpy as np
# Let's go ahead and seed the notebook, for reproducibility
np.random.seed(113)

# Visualizations
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.offline as offline

# Interactive widget
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [2]:
data = pd.read_csv("data/TX_County_By_Year.csv")

In [3]:
data.head()

,COUNTY,COUNTY_ID,YEAR,TOTAL_POPULATION,TOTAL_PILLS,PILLS_PER_CAPITA,TOTAL_OVERDOSE_DEATHS,DEATHS_PER_CAPITA,DEATHS_PER_100K_PEOPLE,PRESCRIPTION_RATE,UNEMPLOYMENT
0,ANDERSON,48001,2006,57386,2209130.0,38.495975,2.5,0.000044,4.356463,95.9,6.2
1,ANDREWS,48003,2006,13195,246600.0,18.688897,0.0,0.000000,0.000000,56.9,3.8
2,ANGELINA,48005,2006,83810,3070975.0,36.642107,2.5,0.000030,2.982938,109.5,4.9
3,ARANSAS,48007,2006,23395,734500.0,31.395597,2.5,0.000107,10.686044,84.9,4.8
4,ARCHER,48009,2006,9063,0.0,0.000000,0.0,0.000000,0.000000,NaN,3.6


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1778 entries, 0 to 1777
Data columns (total 11 columns):
COUNTY                    1778 non-null object
COUNTY_ID                 1778 non-null int64
YEAR                      1778 non-null int64
TOTAL_POPULATION          1778 non-null int64
TOTAL_PILLS               1778 non-null float64
PILLS_PER_CAPITA          1778 non-null float64
TOTAL_OVERDOSE_DEATHS     1778 non-null float64
DEATHS_PER_CAPITA         1778 non-null float64
DEATHS_PER_100K_PEOPLE    1778 non-null float64
PRESCRIPTION_RATE         1309 non-null float64
UNEMPLOYMENT              1778 non-null float64
dtypes: float64(7), int64(3), object(1)
memory usage: 152.9+ KB


In [5]:
data["PRESCRIPTION_RATE"].fillna(value=0, inplace=True)

In [6]:
data.describe()

,COUNTY_ID,YEAR,TOTAL_POPULATION,TOTAL_PILLS,PILLS_PER_CAPITA,TOTAL_OVERDOSE_DEATHS,DEATHS_PER_CAPITA,DEATHS_PER_100K_PEOPLE,PRESCRIPTION_RATE,UNEMPLOYMENT
count,1778.000000,1778.000000,1.778000e+03,1.778000e+03,1778.000000,1778.000000,1778.000000,1778.000000,1778.000000,1778.000000
mean,48254.000000,2009.000000,9.745832e+04,3.055180e+06,27.628911,2.889483,0.000032,3.163661,55.477559,6.621822
std,146.687088,2.000563,3.472073e+05,1.179600e+07,18.509325,15.573713,0.000103,10.312937,45.323831,2.460691
min,48001.000000,2006.000000,6.100000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
25%,48127.000000,2007.000000,6.644750e+03,1.220800e+05,15.005555,0.000000,0.000000,0.000000,0.000000,4.700000
50%,48254.000000,2009.000000,1.824500e+04,4.788050e+05,26.125323,0.000000,0.000000,0.000000,60.200000,6.300000
75%,48381.000000,2011.000000,4.898000e+04,1.855902e+06,38.866253,2.500000,0.000030,2.959815,86.075000,8.000000
max,48507.000000,2012.000000,4.264193e+06,1.830127e+08,119.518327,309.000000,0.002822,282.167043,213.600000,18.600000


In [7]:
# Defining a list of relevant years
years = list(data["YEAR"].unique())

# Defining the columns for the drop-down menu, as well as the bins for the
# legend associated with each column
columns = {"TOTAL_POPULATION": [5000, 10000, 50000, 100000],
           "TOTAL_PILLS": [10000, 100000, 500000, 1000000, 5000000, 10000000],
           "PILLS_PER_CAPITA": [1, 25, 50, 75],
           "TOTAL_OVERDOSE_DEATHS": [1, 10, 25, 50],
           "DEATHS_PER_100K_PEOPLE": [1, 5, 10, 25, 50],
           "PRESCRIPTION_RATE": [1, 50, 75, 100],
           "UNEMPLOYMENT": [5, 6.5, 8, 10]
           }

In [8]:
# Using ipywidget to create dropdown menus for the arguments passed into the
# below function, color_counties
@interact
# Defining our visualization function
# Here, the init values in the function define the dropdown menus
def color_counties(year=years, column=columns.keys()):

    # Creating a dataframe based on the year chosen
    df = data.loc[data["YEAR"] == year]
    
    # Creating our choropleth map
    fig = ff.create_choropleth(
        # Providing the FIPS state and county codes for location
        fips=df["COUNTY_ID"],

        # Determining color in the map, here I'm providing the chosen column
        values=df[column],

        # This will accept state abbreviation or name, expected as a string
        scope=["TX"],

        # Creating bins for the colors and the legend, which I'm giving as the 
        # values of our columns dictionary
        binning_endpoints=columns[column],

        # Outlining the counties in white more obviously
        county_outline={'color': 'rgb(255,255,255)',
                        'width': 0.5},

        # Naming the legend as our column name
        legend_title=column,
    )

    # Setting this to None generates the map on a white background
    fig.layout.template = None
    fig.layout.title = "Texas County Data, " + str(year)
    fig.show()
    
#     offline.plot(fig, filename='TX_county_data'+'.html') 

# Note - this often results in a FutureWarning that I can't seem to avoid
# Says to add sort=True/False but not sure exactly where to add that

interactive(children=(Dropdown(description='year', options=(2006, 2007, 2008, 2009, 2010, 2011, 2012), value=2…